In [81]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px



In [82]:
partCsv= "/home/alextay96/Desktop/new_workspace/DLDataPipeline/data/results/part_perf.csv"
partPerfDf = pd.read_csv(partCsv)
caseAccCsv= "/home/alextay96/Desktop/new_workspace/DLDataPipeline/data/results/acc_perf.csv"
caseAccDf = pd.read_csv(caseAccCsv)


In [83]:
allParts = [x.replace("gt_", "") for x in partPerfDf.columns if "Unname" not in x and "gt_" in x]
allParts

['vision_bumper_rear',
 'vision_misc',
 'vision_rear_compartment',
 'vision_rear_panel',
 'vision_tail_lamp_lh',
 'vision_tail_lamp_rh',
 'vision_non_external',
 'vision_bonnet',
 'vision_bumper_front',
 'vision_engine',
 'vision_front_panel',
 'vision_grille',
 'vision_headlamp_lh',
 'vision_headlamp_rh',
 'vision_windscreen_front',
 'vision_door_front_rh',
 'vision_door_rear_rh',
 'vision_rear_quarter_rh',
 'vision_wheel',
 'vision_door_front_lh',
 'vision_door_rear_lh',
 'vision_fender_front_lh',
 'vision_fender_front_rh',
 'vision_rear_quarter_lh']

In [84]:
partMetrics = []
for part in allParts:
    gtCol = partPerfDf[f"gt_{part}"].values
    predCol = partPerfDf[f"pred_{part}"].values
    # posGt = gtCol[gtCol == 1]
    posGtCount = np.count_nonzero(gtCol == 1)
    negGtCount = np.count_nonzero(gtCol == 0)

    tp =  np.count_nonzero(predCol[gtCol == 1] == 1) / posGtCount
    tn = np.count_nonzero(predCol[gtCol == 0] == 0 ) / negGtCount
    fp =  np.count_nonzero(predCol[gtCol == 0] == 1) / negGtCount
    fn =  np.count_nonzero(predCol[gtCol == 1] == 0) / posGtCount
    assert 0.99 < tp + fn < 1.01
    assert 0.99 < tn + fp < 1.01
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    acc = (tp + tn) / (tp + tn + fp + fn)
    partMetrics.append(
        {
            "part" : part,
            "tp": tp,
            "tn": tn,
            "fn": fn,
            "fp": fp,
            "precision": precision,
            "recall": recall,
            "acc" : acc,
            "gt_pos_count" : posGtCount,
            "gt_neg_count" : negGtCount,
            "gt_pos_ratio" : posGtCount / (posGtCount + negGtCount)

        }
    )

partEvalMetrics = pd.json_normalize(partMetrics)
partEvalMetrics.to_csv("/home/alextay96/Desktop/new_workspace/DLDataPipeline/data/results/part_metrics.csv")
    

In [85]:
def genVisMulitlabel(multilabelPartDf, metricName):
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=multilabelPartDf["part"], y=multilabelPartDf[metricName],
        text = multilabelPartDf[metricName]
        ))
    avgVal = multilabelPartDf[metricName].mean()
    fig.add_hline(y=avgVal, line_width=2, line_color="red")
    fig.update_layout(
        yaxis = dict(
            tickmode = 'array',
            tickvals = [np.format_float_positional(avgVal, 2), 0.3, 0.8],
        )
    )
    fig.update_layout(
            title=go.layout.Title(
                text=f"{metricName}",
                x=0.5
            ))
    
    fig.show()
   

In [86]:
genVisMulitlabel(partEvalMetrics, "tp")

In [87]:
genVisMulitlabel(partEvalMetrics, "tn")

In [88]:
genVisMulitlabel(partEvalMetrics, "acc")

In [89]:
genVisMulitlabel(partEvalMetrics, "gt_pos_ratio")

In [90]:
meanVal = caseAccDf["subset_acc"].mean()
# medianVal = multilabelCaseDf["subset_acc"].median()
firstQuarter = np.quantile(caseAccDf["subset_acc"].values, 0.25)
thirdQuarter = np.quantile(caseAccDf["subset_acc"].values, 0.75)

fig = px.histogram(caseAccDf, x="subset_acc")
fig.add_vline(x=meanVal, line_width=2, line_color="red")
# fig.add_vline(x=medianVal, line_width=2, line_color="blue")
fig.add_vline(x=firstQuarter, line_width=2, line_color="orange")
fig.add_vline(x=thirdQuarter, line_width=2, line_color="green")

fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [meanVal, firstQuarter, thirdQuarter],
    )
)

In [91]:
corr = partEvalMetrics["tp"].corr(partEvalMetrics["gt_pos_ratio"])
corr

0.7619949712571135

In [92]:
corr = partEvalMetrics["tn"].corr(partEvalMetrics["gt_pos_ratio"])
corr

-0.9236564622730535

In [93]:
corr = partEvalMetrics["acc"].corr(partEvalMetrics["gt_pos_ratio"])
corr

0.2301585938017402